**Gruppemedlemmer:** Sindre Vie Jørgensen, Viktor Mikalsen, Even Tobias Storli og Kasper Voogt Arvesen
# Prosjekt 2: TMA4320
* **Innleveringsfrist:** Mandag 9. mars, 2020, kl 1400



In [1]:
import matplotlib.pyplot as plt
import numpy as np
import numpy.linalg as la
import scipy as sp
import random as rn
from numba import jit, njit, vectorize
from spirals import get_data_spiral_2d

###########################################################################################################
                        #Variabler
    
h = 0.1

**Hjelpefunksjoner: Def av variabler for $Y_0$ og $Y_{k+1}$**

In [2]:
##############################################################################
                    #Def av variabler for Y_0 og Y_k1

def eta(x):
    eta = 1/2 * (1 + np.tanh(x/2))
    return eta

def sigma(x):              #Tror denne er unødvendig vi trenger denne ettersom sigma kun er np.tanh
    sigma = np.tanh(x)
    return sigma

def big_z(eta, mat_y, omega, my, d):        #funk er ikke ferdig, noe mer må gjøres med mat_y
    big_z = eta(x)*(mat_y.transpose()*omega + my*np.eye(d, k=0))    #numpy.eye(a, b) lager en axa matrise hvor subdiagonal/diag b er 1 og resten 0. Nå er diagonalen 1.
    return big_z

def big_j(big_z, c):            #Fungerer for numpy arrays
    c = -1*c
    big_j = 0.5*la.norm(np.add(big_z, c))**2
    return big_j



**Definisjon av $Y_0$ og $Y_{k+1}$**